#
<FONT size="10pt"> <center>Projet perso : Les Champignons</center></FONT>. 

- **Scraping des données textuelles et images des champignons du site internet : www.guidedeschampignons.com**
- **Utilisation des librairies Python :**
    - Selenium pour l'extraction des données.
    - Requests pour l'enregistrement des photos.
- **Apprentissage de l'extraction et de l'enregistrement de photos en scraping.**

# 1 : Librairies et Options

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import re
import time
from tqdm import tqdm
import requests

import os
os.environ['WDM_LOG_LEVEL'] = '0' # Permet de masquer les informations de scraping inutiles qui encombrent la page (a faire quand il y a pas de pb avec le code)

# 2 : Scraping des champignons

In [2]:
# Ouverture de chrome et de la page à scraper :
driver = webdriver.Chrome()
driver.get("https://www.guidedeschampignons.com/tous-les-champignons/")
driver.maximize_window()

# Autorisation des données :
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div[2]/div[1]/div[2]/div[2]/button[2]/p'))).click()

# Création des listes pour stocker les données :
Noms, Comestibles, Chapeaux, Lames, Pieds, Chairs, Odeurs, Saveurs, Habitats, Saisons, NomPhotos = [], [], [], [], [], [], [], [], [], [], []

# Fonction pour récupérer le texte en utilisant une approche try-except
def get_text(driver, xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except:
        return ""

# Chargement des pages :
for page in range(2, 5):
    link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,('//*[@id="index-5902226"]/div[2]/div/ul/li[{}]/*'.format(page)))))
    driver.execute_script("arguments[0].scrollIntoView()",link) # Scrolling until he find the path
    link.click() 
    time.sleep(5)

    # Compte le nombre de champignons sur cette page :
    NbChampignons = len(driver.find_elements(By.XPATH, "//*[@class='t-entry-visual-overlay']"))

    # Cliquer sur les champignons :
    for pos in tqdm(range(1, NbChampignons + 1)):
        
        # Ouverture d'un des champignons :    
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,('/html/body/div[3]/div/div[2]/div/div/div/article/div/div/div/div[2]/div/div/div/div/div/div/div/div/div[1]/div/div[{}]/div/div[2]/div/div/h2/a'.format(pos)))))
        driver.execute_script("arguments[0].scrollIntoView()",link) # Scrolling until he find the path
        link.click() 
        time.sleep(5)

        # Récupérer les informations utiles :
        Nom = get_text(driver, '/html/body/div[3]/div/div[2]/div/div/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div/h1')
        Noms.append(Nom)

        Comestibles.append(get_text(driver, '/html/body/div[3]/div/div[2]/div/div/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/p[2]/strong') or get_text(driver, '/html/body/div[3]/div/div[2]/div/div/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/p[2]/b'))

        Chapeaux.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'CHAPEAU')]]") or get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'FORME')]]"))

        Lames.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'LAMES')]]") or get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'PORES')]]") or 'Aucuns : Aucuns')

        Pieds.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'PIED')]]") or 'PIED : Non Définie')

        Chairs.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'CHAIR')]]") or 'CHAIR : Non Définie')

        Odeurs.append(get_text(driver, "//body//div[@class='uncont']//div/p[*[contains(text(), 'ODEUR')]]") or get_text(driver, "//body//div[@class='uncont']//div/p[*/*[contains(text(), 'ODEUR')]]"))

        Saveurs.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong/b[contains(text(), 'SAVEUR')]]") or get_text(driver, "//body//div[@class='uncont']//div/p[b[contains(text(), 'SAVEUR')]]") or 'SAVEUR : Non Définie')

        Habitats.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'HABITAT')]]"))

        Saisons.append(get_text(driver, "//body//div[@class='uncont']//div/p[strong[contains(text(), 'SAISON')]]"))

        # Récupérer les images :
        path = 'C:\\Users\\Johan\\Formation Data Science\\Champignons\\PhotosChampignons\\'
        image_tag = driver.find_elements(By.CLASS_NAME, 'woocommerce-product-gallery__image-first__img.wp-post-image')
        image_url = [img.get_attribute('src') for img in image_tag]

        response = requests.get(image_url[0], stream=True)

        NomPhoto = Nom.capitalize().replace(' ', '') + '.jpg'
        NomPhotos.append(NomPhoto)

        with open(path + NomPhoto, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)

        # Revenir à la page précédente :
        driver.back()

# Fermer la fenêtre :
driver.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [04:36<00:00, 10.25s/it]


In [3]:
# Création de la DF finale :

data = {
    'Nom': Noms,
    'Comestibilite': Comestibles,
    'FormeChapeau': [chapeau.split(':')[1] for chapeau in Chapeaux],
    'LamePore': [lame.split(':')[0] for lame in Lames],
    'LamePoreInfo': [lame.split(':')[1] for lame in Lames],
    'Pied': [pied.split(':')[1] for pied in Pieds],
    'Chair': [chair.split(':')[1] for chair in Chairs],
    'Odeur': [odeur.split(':')[1] for odeur in Odeurs],
    'Saveur': [saveur.split(':')[1] for saveur in Saveurs],
    'Habitat': [habitat.split(':')[1] for habitat in Habitats],
    'Saison': [saison.split(':')[1] for saison in Saisons],
    'Photos': NomPhotos
}

df = pd.DataFrame(data)
df

,Nom,Comestibilite,FormeChapeau,LamePore,LamePoreInfo,Pied,Chair,Odeur,Saveur,Habitat,Saison,Photos
0,AGARIC AUGUSTE,À rejeter,"5 à 25 cm, couvert de mèches brun-roux sur fo...",LAMES,"Libres, crème puis gris-rose et enfin brunes.","Blanc, élancé (6 à 20 cm), légèrement en mass...","Blanche, jaunissant lentement à la coupe, ros...","Agréable, d’amande amère",Douce,"Bois clairs de feuillus ou de conifères, lisi...",Juillet > Octobre,Agaricauguste.jpg
1,AGARIC DES JACHÈRES,Excellent comestible,"5 à 15 cm, lisse, blanc puis jaunissant en vi...",LAMES,"Libres, serrées, gris-rose très pâle puis bru...","5 à 15 cm, élancé, avec un large anneau blanc...","Blanche, ferme",Anisée,Douce,"Surtout sous feuillus, clairières, lisières, ...",Juin > Novembre,Agaricdesjachères.jpg
2,AGARIC JAUNISSANT,Toxique,"2 à 15 cm, blanc à grisâtre pâle, lisse, souv...",LAMES,"Libres, serrées, gris-rose pâle puis brunes","3 à 15 cm, avec un bulbe net et un anneau flo...","Blanche, jaunissant fortement à la base du pied","Désagréable, d’encre ou de phénol","Désagréable, iodée","Prairies, jardins ou bois clairs",Mai > Novembre,Agaricjaunissant.jpg
3,AGARIC SYLVICOLE,Bon comestible,"3 à 12 cm, blanc à crème, centre plus ou moin...",LAMES,"Libres, blanc rosâtre puis roses et enfin bru...","Blanc, en massue ou bulbeux, 2 à 15 cm, avec ...","Blanche, ferme","Forte, d’anis",Douce,"Bois clairs, surtout de feuillus",Août > Octobre,Agaricsylvicole.jpg
4,AMANITE CITRINE,À rejeter,"4 à 10 cm, jaune citron pâle, avec des plaque...",LAMES,"Libres, blanches à jaune citron pâle","3 à 15 cm, jaunâtre, avec un anneau mince et ...",Blanche à jaune citron pâle,"Forte, de pomme de terre crue ou de radis",Douce,Forêts de feuillus et de conifères,Juillet > Octobre,Amanitecitrine.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
214,VESSE-DE-LOUP EN FORME DE POIRE,Comestible jeune,Boule d’abord blanche puis beige-ocre (1 à 3 ...,Aucuns,Aucuns,"Long, bien formé, de 1 à 3 cm",Blanche puis brun olive et molle. Pulvérulent...,Faible,Douce,En touffe sur le bois mort,Août > Novembre,Vesse-de-loupenformedepoire.jpg
215,VESSE-DE-LOUP GÉANTE,Bon comestible,"Grosse boule (15 à 65 cm) blanche et lisse, p...",Aucuns,Aucuns,Absent,"Épaisse, blanche puis jaune à olivâtre. Pulvé...","Faible, puis désagréable",Douce,Champs et prairies,Juin > Novembre,Vesse-de-loupgéante.jpg
216,VESSE-DE-LOUP HÉRISSON,Comestible très jeune,Boule (2 à 6 cm) couverte de longs aiguillons...,Aucuns,Aucuns,"Court (0,5 à 2 cm), beige-ocre, couvert d’aig...",Blanche puis brun olivâtre et molle. Pulvérul...,"Faible, de bolet",Douce,Surtout sous les hêtres,Juillet > Novembre,Vesse-de-louphérisson.jpg
217,VESSE-DE-LOUP PERLÉE,Comestible jeune,Boule blanche (2 à 7 cm) recouverte de fins a...,Aucuns,Aucuns,"Long, de 1 à 4 cm, bien formé","Cotonneuse, blanche puis olivâtre et pulvérul...",Faible,Douce,Forêts de feuillus et de conifères,Mai > Novembre,Vesse-de-loupperlée.jpg


In [4]:
# Exportation du fichier en csv : 
df.to_csv('ChampignonsExtract.csv', index = False)